## Builting Functions

### Initial Setup

In [144]:
import torch

# Print the CUDA version PyTorch is built with
print("Built CUDA Version:", torch.version.cuda)

# Print the CUDA version runtime (if CUDA is available)
if torch.cuda.is_available():
    print("CUDA Runtime Version:", torch._C._cuda_getCompiledVersion())
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available.")


Built CUDA Version: None
CUDA is not available.


In [145]:
class_id_to_name = {
    0:  ('road', [28, 42, 168]),
    1:  ('pool', [0, 50, 89]),
    2:  ('vegetation', [107, 142, 35]),
    3:  ('roof', [70, 70, 70]),
    4:  ('wall', [102, 102, 156]),
    5:  ('window', [254, 228, 12]),
    6:  ('person', [255, 22, 96]),
    7:  ('dog', [102, 51, 0]),
    8:  ('car', [9, 143, 150]),
    9:  ('bicycle', [119, 11, 32]),
    10: ('tree', [51, 51, 0]),
    11: ('truck', [160, 160, 60]),   # added truck
    12: ('bus', [200, 80, 80]),      # added bus
    13: ('vehicle', [200, 80, 80]),      # added bus
}

### Install packages

In [146]:
# !pip install numpy
# !pip install opencv-python
# !pip install pillow
# !pip install matplotlib
# !pip install tqdm
# !pip install scikit-learn
# !pip install torch torchvision
# !pip install ultralytics

In [147]:
# !pip uninstall torch torchvision torchaudio
# !pip cache purge  # clean out pip's install cache
# !pip install torch torchvision torchaudio --force-reinstall

In [148]:
# Core packages
import os
import shutil
import json
import zipfile
import xml.etree.ElementTree as ET
from pathlib import Path

# Math and array handling
import numpy as np
from sklearn.model_selection import train_test_split

# Image and visualization
import cv2
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Progress bar
from tqdm.auto import tqdm

# Deep Learning Frameworks
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.models.segmentation as segmentation
import torchvision.transforms as transforms

# Object Detection and Segmentation
from ultralytics import YOLO

from torch.utils.data import DataLoader
import gc

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# Automatically use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

%matplotlib inline

import random
import gdown

from collections import defaultdict
from glob import glob

from PIL import Image

# save this as split_uavdt_train_val.py

import os
import shutil
from glob import glob
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

import os
from PIL import Image

import os
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

import os
import random
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
from ultralytics import YOLO

import os
import pandas as pd
# from .autonotebook import tqdm as notebook_tqdm

import os
from collections import defaultdict

### Download Datsets 

In [149]:
def semantic_drone_dataset_download(gdrive_url, extract_to="extracted"):
    # Convert shared drive URL to direct download URL
    file_id = gdrive_url.split("/d/")[1].split("/")[0]
    download_url = f"https://drive.google.com/uc?id={file_id}"

    # Create output folder
    os.makedirs(extract_to, exist_ok=True)

    zip_path = os.path.join(extract_to, "downloaded.zip")

    print("[INFO] Downloading ZIP from Google Drive...")
    gdown.download(download_url, zip_path, quiet=False)

    print("[INFO] Extracting ZIP...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

    # Optionally, remove the ZIP file after extraction
    os.remove(zip_path)

    print(f"[DONE] Extracted files to: {extract_to}")

def uavdt_dataset_download(gdrive_url, extract_to="extracted"):
    # Convert shared drive URL to direct download URL
    file_id = gdrive_url.split("/d/")[1].split("/")[0]
    download_url = f"https://drive.google.com/uc?id={file_id}"

    # Create output folder
    os.makedirs(extract_to, exist_ok=True)

    zip_path = os.path.join(extract_to, "downloaded.zip")

    print("[INFO] Downloading ZIP from Google Drive...")
    gdown.download(download_url, zip_path, quiet=False)

    print("[INFO] Extracting ZIP...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

    # Optionally, remove the ZIP file after extraction
    os.remove(zip_path)

    print(f"[DONE] Extracted files to: {extract_to}")



### Convert Two Datsets into yolo format

In [150]:
# ----------------------------
# Parse polygon and convert to YOLO bbox
# ----------------------------
# Semantic drone datasets 
def parse_yolo_style_bbox_from_xml(xml_path, class_id_to_name):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    bboxes = []
    for obj in root.findall('object'):
        class_name = obj.find('name').text
        if class_name in [value[0] for value in class_id_to_name.values()]:
            polygon = obj.find('polygon')
            if polygon is not None:
                points = polygon.findall('pt')
                coords = [(float(pt.find('x').text), float(pt.find('y').text)) for pt in points]
                x_min = min(coord[0] for coord in coords)
                y_min = min(coord[1] for coord in coords)
                x_max = max(coord[0] for coord in coords)
                y_max = max(coord[1] for coord in coords)
                bboxes.append(((x_min, y_min), (x_max, y_max), class_name))
    return bboxes


# ----------------------------
# Save YOLO-format txt
# ----------------------------
def save_yolo_format(image_id, bboxes, image_width, image_height, output_path, class_id_to_name):
    with open(output_path, 'w') as f:
        for (x_min, y_min), (x_max, y_max), class_name in bboxes:
            class_id = next(cid for cid, (name, _) in class_id_to_name.items() if name == class_name)
            x_center = (x_min + x_max) / 2 / image_width
            y_center = (y_min + y_max) / 2 / image_height
            width = (x_max - x_min) / image_width
            height = (y_max - y_min) / image_height
            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")


# ----------------------------
# Convert dataset (YOLO only)
# ----------------------------
def convert_fulldataset_yolo_only(dataset_path, output_dir, class_id_to_name):
    image_ids = [img.split('.')[0] for img in os.listdir(f"{dataset_path}/images") if img.endswith(".jpg")]

    os.makedirs(f"{output_dir}/images", exist_ok=True)
    os.makedirs(f"{output_dir}/labels", exist_ok=True)

    for image_id in tqdm(image_ids, desc="Converting to YOLO"):
        img_path = f"{dataset_path}/images/{image_id}.jpg"
        bbox_xml_path = f"{dataset_path}/gt/bounding_box/label_me_xml/{image_id}.xml"
        semantic_xml_path = f"{dataset_path}/gt/semantic/label_me_xml/{image_id}.xml"

        if not os.path.exists(img_path):
            print(f"[WARNING] Image not found: {img_path}, skipping...")
            continue

        try:
            bboxes1 = parse_yolo_style_bbox_from_xml(bbox_xml_path, class_id_to_name)
            bboxes2 = parse_yolo_style_bbox_from_xml(semantic_xml_path, class_id_to_name)
            all_bboxes = bboxes1 + bboxes2
        except Exception as e:
            print(f"[WARNING] Skipping image {image_id} due to parse error: {e}")
            continue

        try:
            image = Image.open(img_path)
            image_np = np.array(image)
        except Exception as e:
            print(f"[WARNING] Could not load image {image_id}: {e}")
            continue

        # Save image
        image.save(f"{output_dir}/images/{image_id}.jpg")

        # Save YOLO labels
        yolo_annotation_path = f"{output_dir}/labels/{image_id}.txt"
        save_yolo_format(image_id, all_bboxes, image_np.shape[1], image_np.shape[0], yolo_annotation_path, class_id_to_name)

    print("✅ YOLO-format annotation conversion complete!")

In [151]:
import os
import shutil
import cv2
from glob import glob
from collections import defaultdict
from sklearn.model_selection import train_test_split

# 🧠 Map UAVDT class to extended class_id_to_name
uavdt_to_extended = {
    0: 8,   # car
    1: 11,  # truck
    2: 12,  # bus
    3: 13
}

# === Function to Convert Single Annotation to YOLO Format ===
def convert_annotation(anno_path, label_path, image_path, stats):
    if not os.path.exists(image_path):
        stats["missing_image"] += 1
        return

    try:
        img = cv2.imread(image_path)
        height, width = img.shape[:2]
    except:
        stats["missing_image"] += 1
        return

    with open(anno_path, 'r') as fin, open(label_path, 'w') as fout:
        for line in fin:
            parts = line.strip().split(',')
            if len(parts) < 8:
                stats["malformed"] += 1
                continue

            try:
                x, y, w, h = map(float, parts[0:4])
                original_cls = int(parts[5])

                # 🔁 Convert original class to extended class
                if original_cls not in uavdt_to_extended:
                    stats["skipped"][original_cls] += 1
                    continue

                cls = uavdt_to_extended[original_cls]

                x_center = (x + w / 2) / width
                y_center = (y + h / 2) / height
                w /= width
                h /= height

                if not (0 <= x_center <= 1 and 0 <= y_center <= 1 and w > 0 and h > 0):
                    stats["skipped"][cls] += 1
                    continue

                fout.write(f"{cls} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")
                stats["converted"] += 1
            except Exception:
                stats["malformed"] += 1
                continue

            stats["total"] += 1

# === Step 1: Convert UAVDT annotations to YOLO format ===
def convert_dataset(root_dir):
    annotation_paths = glob(os.path.join(root_dir, "M*/annotations/*.txt"))
    total_files = len(annotation_paths)

    stats = {
        "total": 0,
        "converted": 0,
        "malformed": 0,
        "missing_image": 0,
        "skipped": defaultdict(int)
    }

    print(f"🔄 Converting {total_files} annotation files to YOLO format...")

    for anno_path in tqdm(annotation_paths, desc="Converting", unit="file"):
        sequence_dir = os.path.dirname(os.path.dirname(anno_path))  # Mxxxx
        file_name = os.path.basename(anno_path)

        label_dir = os.path.join(sequence_dir, "labels")
        os.makedirs(label_dir, exist_ok=True)

        label_path = os.path.join(label_dir, file_name)

        # Construct image path
        image_name = file_name.replace(".txt", ".jpg")
        image_path = os.path.join(sequence_dir, "images", image_name)

        convert_annotation(anno_path, label_path, image_path, stats)

    print("\nConversion complete.")
    print(f"Total boxes:     {stats['total']}")
    print(f"Converted boxes: {stats['converted']}")
    print(f"Skipped boxes:   {sum(stats['skipped'].values())}")
    for cls, count in sorted(stats["skipped"].items()):
        print(f"   - Skipped class {cls}: {count}")
    print(f"Malformed lines: {stats['malformed']}")
    print(f"Missing images: {stats['missing_image']}")

# === Step 2: Copy to train/val structure ===
def copy_split_sequences(src_root, dst_root, train_ratio=0.8):
    all_sequences = sorted(glob(os.path.join(src_root, "M*")))
    train_seqs, val_seqs = train_test_split(all_sequences, train_size=train_ratio, random_state=42)

    for split_name, split_list in zip(['train', 'val'], [train_seqs, val_seqs]):
        for seq_path in tqdm(split_list, desc=f"Copying {split_name}"):
            images_src = os.path.join(seq_path, "images")
            labels_src = os.path.join(seq_path, "labels")

            images_dst = os.path.join(dst_root, split_name, "images")
            labels_dst = os.path.join(dst_root, split_name, "labels")

            os.makedirs(images_dst, exist_ok=True)
            os.makedirs(labels_dst, exist_ok=True)

            for img_file in glob(os.path.join(images_src, "*.jpg")):
                shutil.copy(img_file, os.path.join(images_dst, os.path.basename(img_file)))

            for label_file in glob(os.path.join(labels_src, "*.txt")):
                shutil.copy(label_file, os.path.join(labels_dst, os.path.basename(label_file)))

    print("\nDataset split into 'train/' and 'val/' with images and YOLO labels.")


### Convert into train and Val sets

In [152]:
# Semantic Drone Datasets
def move_files(file_list, 
               source_image_dir, 
               source_annotation_dir,
               target_image_dir, 
               target_annotation_dir):
    
    os.makedirs(target_image_dir, exist_ok=True)
    os.makedirs(target_annotation_dir, exist_ok=True)

    for image_id in tqdm(file_list, desc=f"Moving to {os.path.basename(os.path.dirname(target_image_dir))}"):
        image_path = os.path.join(source_image_dir, f"{image_id}.jpg")
        annotation_path = os.path.join(source_annotation_dir, f"{image_id}.txt")

        target_image_path = os.path.join(target_image_dir, f"{image_id}.jpg")
        target_annotation_path = os.path.join(target_annotation_dir, f"{image_id}.txt")

        if os.path.exists(image_path):
            shutil.copy(image_path, target_image_path)
        else:
            print(f"[Warning] Missing image: {image_path}")

        if os.path.exists(annotation_path):
            shutil.copy(annotation_path, target_annotation_path)
        else:
            print(f"[Warning] Missing annotation: {annotation_path}")

def split_and_move_dataset(source_base_dir="./datasets/semantic_yolo",
                           target_base_dir="./datasets/new_dataset_yolo_split",
                           split_ratio=0.8,
                           seed=42):
    random.seed(seed)

    image_dir = os.path.join(source_base_dir, "images")
    label_dir = os.path.join(source_base_dir, "labels")

    image_ids = [os.path.splitext(f)[0] for f in os.listdir(image_dir) if f.endswith(".jpg")]
    random.shuffle(image_ids)

    split_idx = int(len(image_ids) * split_ratio)
    train_ids = image_ids[:split_idx]
    val_ids = image_ids[split_idx:]

    # Train
    move_files(train_ids,
               source_image_dir=image_dir,
               source_annotation_dir=label_dir,
               target_image_dir=os.path.join(target_base_dir, "train/images"),
               target_annotation_dir=os.path.join(target_base_dir, "train/labels"))

    # Val
    move_files(val_ids,
               source_image_dir=image_dir,
               source_annotation_dir=label_dir,
               target_image_dir=os.path.join(target_base_dir, "val/images"),
               target_annotation_dir=os.path.join(target_base_dir, "val/labels"))

    print(f"\n[✓] Dataset split completed: {len(train_ids)} train / {len(val_ids)} val samples")


### Normalize Labels

In [153]:
def normalize_label_file(label_file, img_width, img_height):
    """
    Normalize the label coordinates in a label file to ensure they are within [0, 1] range.
    """
    with open(label_file, 'r') as f:
        lines = f.readlines()
    
    with open(label_file, 'w') as f:
        for line in lines:
            parts = line.strip().split()
            class_id = int(parts[0])
            x_center, y_center, width, height = map(float, parts[1:])
            
            # Normalize coordinates to ensure they are within the range [0, 1]
            x_center = min(1.0, max(0.0, x_center))
            y_center = min(1.0, max(0.0, y_center))
            width = min(1.0, max(0.0, width))
            height = min(1.0, max(0.0, height))

            # Write normalized values back to file
            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")


def get_image_size(img_path):
    """
    Get the width and height of the image to normalize the coordinates properly.
    """
    with Image.open(img_path) as img:
        return img.size  # returns (width, height)


def normalize_all_labels(labels_dir, img_dir):
    """
    Normalize all label files in the specified directory.
    """
    for label_file in tqdm(os.listdir(labels_dir)):
       
        if label_file.endswith('.txt'):  # Process only label files
            label_path = os.path.join(labels_dir, label_file)
            img_path = os.path.join(img_dir, label_file.replace('.txt', '.jpg'))  # Assuming JPG images
            if os.path.exists(img_path):
                # Get image dimensions to normalize the labels
                img_width, img_height = get_image_size(img_path)
                # print(f"Normalizing {label_file}...")
                normalize_label_file(label_path, img_width, img_height)
            else:
                print(f"Warning: Image for label {label_file} not found!")
    print("Normalize Complete")



### Training v8 model functions

In [154]:
def train_yolo(data_yaml, epochs, imgsz, batch, name, model, dir):
    if(model):
        model = YOLO(dir)
    else:
        model = YOLO("yolov8m.pt")

    model.train(
        data=data_yaml,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch,
        name=name,
        project="runs/train",
        patience=30,  # Early stopping
        augment=True,  # Apply augmentations
        degrees=10,  # Image rotation
        scale=0.5,  # Scale range
        flipud=0.2,  # Vertical flip
        fliplr=0.5,  # Horizontal flip
        hsv_h=0.015,  # Hue augmentation
        hsv_s=0.7,  # Saturation augmentation
        hsv_v=0.4,  # Value augmentation
        mosaic=1.0,  # Mosaic augmentation
        mixup=0.2,  # Mixup augmentation
        lr0=0.01,  # Initial learning rate (you can tune this)
        lrf=0.01,  # Learning rate final factor (for cosine annealing)
        verbose=True  # Print progress
    )


### Print val metrics

In [155]:
import json
from ultralytics import YOLO

def load_yolo_model(model_path):
    return YOLO(model_path)

def run_model_validation(model):
    return model.val()

def extract_per_class_metrics(results):
    """
    Extracts mAP@0.5:0.95 per class from results.
    NOTE: Only mAP@0.5:0.95 is available via `results.box.maps`
    """
    per_class_metrics = {}
    if hasattr(results.box, 'maps') and results.box.maps is not None:
        maps = results.box.maps  # This is a NumPy array [num_classes]
        for i, name in results.names.items():
            per_class_metrics[name] = {
                "class_id": i,
                "mAP@0.5:0.95": round(float(maps[i]), 4)
            }
    else:
        print("⚠️ No per-class mAP@0.5:0.95 data found.")
    return per_class_metrics

def save_metrics_to_json(metrics, output_path):
    with open(output_path, "w") as f:
        json.dump(metrics, f, indent=4)
    print(f"✅ Saved per-class metrics to {output_path}")

def evaluate_and_save_metrics(model_path, output_json_path="per_class_metrics.json"):
    model = load_yolo_model(model_path)
    results = run_model_validation(model)
    metrics = extract_per_class_metrics(results)
    save_metrics_to_json(metrics, output_json_path)



In [156]:
import json

def print_per_class_metrics(json_path="per_class_metrics.json"):
    with open(json_path, "r") as f:
        metrics = json.load(f)
    
    print("📊 Per-Class mAP@0.5:0.95 Metrics:\n")
    print(f"{'Class Name':<15} {'Class ID':<10} {'mAP@0.5:0.95':<15}")
    print("-" * 40)
    
    for name, data in metrics.items():
        print(f"{name:<15} {data['class_id']:<10} {data['mAP@0.5:0.95']:<15}")


### Find best model path after training 

In [157]:
def find_best_model(base_dir='runs_yolo/'):
    best_paths = list(Path(base_dir).rglob('best.pt'))
    if not best_paths:
        raise FileNotFoundError("No 'best.pt' file found in the 'runs/' directory.")
    
    # Optionally, sort by latest modified time
    best_paths.sort(key=lambda p: p.stat().st_mtime, reverse=True)
    
    print(f"✅ Found best.pt at: {best_paths[0]}")
    return str(best_paths[0])


### Prediction on vdieos

In [158]:

# ========== FRAME PROCESSING ==========
def process_frame_1(frame, yolo_model, w, h, class_id_to_name):
    annotated = frame.copy()
    results = yolo_model(annotated, verbose=False)[0]
    boxes = results.boxes.xyxy.cpu().numpy()
    class_ids = results.boxes.cls.cpu().numpy()

    for box, cls_id in zip(boxes, class_ids):
        x1, y1, x2, y2 = map(int, box)
        class_name, color = class_id_to_name[int(cls_id)]
        cv2.rectangle(annotated, (x1, y1), (x2, y2), color, 2)
        cv2.putText(annotated, class_name, (x1, max(y1 - 10, 10)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)

    return annotated, boxes, class_ids

# ========== VIDEO CAPTURE ==========
def setup_video_capture_1(video_path):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    return cap, total_frames, fps, w, h

# ========== MAIN FUNCTION ==========
def videos_predictions(yolo_weights_path, class_id_to_name, video_dir='videos', output_base='./datatsets/opt', max_frames=None):
    yolo_model = YOLO(yolo_weights_path)

    image_out_dir = os.path.join(output_base, 'images')
    label_out_dir = os.path.join(output_base, 'labels')
    output_video_dir = os.path.join(output_base, 'output')

    os.makedirs(image_out_dir, exist_ok=True)
    os.makedirs(label_out_dir, exist_ok=True)
    os.makedirs(output_video_dir, exist_ok=True)

    for video_file in tqdm(sorted(os.listdir(video_dir))):
        if not video_file.lower().endswith(".mp4"):
            continue

        video_id = os.path.splitext(video_file)[0]
        video_path = os.path.join(video_dir, video_file)
        output_video_path = os.path.join(output_video_dir, f"{video_id}.mp4")

        print(f"\n========== STARTED: {video_id} ==========")
        cap, total_frames, fps, w, h = setup_video_capture_1(video_path)
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        writer = cv2.VideoWriter(output_video_path, fourcc, fps, (w, h))

        frame_count = 0
        pbar = tqdm(total=max_frames if max_frames else total_frames, desc=video_id)

        while True:
            ret, frame = cap.read()
            if not ret or (max_frames and frame_count >= max_frames):
                break

            annotated_bgr, boxes, class_ids = process_frame_1(frame, yolo_model, w, h, class_id_to_name)

            # ✅ Save original image
            img_filename = f'{video_id}_{frame_count:04d}.jpg'
            img_path = os.path.join(image_out_dir, img_filename)
            cv2.imwrite(img_path, frame)

            # ✅ Save YOLO-format label
            label_filename = f'{video_id}_{frame_count:04d}.txt'
            label_path = os.path.join(label_out_dir, label_filename)
            with open(label_path, 'w') as f:
                for box, cls_id in zip(boxes, class_ids):
                    x1, y1, x2, y2 = box
                    w_box = x2 - x1
                    h_box = y2 - y1
                    cx = x1 + w_box / 2
                    cy = y1 + h_box / 2
                    f.write(f"{int(cls_id)} {cx/w:.6f} {cy/h:.6f} {w_box/w:.6f} {h_box/h:.6f}\n")

            writer.write(annotated_bgr)
            frame_count += 1
            pbar.update(1)

        cap.release()
        writer.release()
        pbar.close()
        print(f"DONE: {video_id} — Processed {frame_count} frames")


In [159]:
import os
import cv2
import numpy as np
from ultralytics import YOLO

# ========== FRAME PROCESSING ==========
def process_frame(frame, yolo_model, w, h, class_id_to_name, valid_class_ids, conf_threshold=0.5):
    annotated = frame.copy()
    results = yolo_model(annotated, verbose=False)[0]

    # Filter by confidence
    mask = results.boxes.conf > conf_threshold
    boxes = results.boxes.xyxy[mask].cpu().numpy()
    class_ids = results.boxes.cls[mask].cpu().numpy()
    confs = results.boxes.conf[mask].cpu().numpy()

    filtered_boxes, filtered_ids, filtered_confs = [], [], []

    for box, cls_id, conf in zip(boxes, class_ids, confs):
        if int(cls_id) in valid_class_ids:
            filtered_boxes.append(box)
            filtered_ids.append(cls_id)
            filtered_confs.append(conf)

            x1, y1, x2, y2 = map(int, box)
            class_name, color = class_id_to_name[int(cls_id)]
            label = f"{class_name} {conf:.2f}"
            cv2.rectangle(annotated, (x1, y1), (x2, y2), color, 2)
            cv2.putText(annotated, label, (x1, max(y1 - 10, 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)

    return annotated, filtered_boxes, filtered_ids

# ========== VIDEO CAPTURE ==========
def setup_video_capture(video_path):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    return cap, total_frames, fps, w, h

# ========== MAIN FUNCTION ==========
def process_all_videos(yolo_weights_path, class_id_to_name, underrepresented_class_ids,
                       video_dir='videos', output_base='./datasets/opt', max_frames=None):
    yolo_model = YOLO(yolo_weights_path)

    image_out_dir = os.path.join(output_base, 'images')
    label_out_dir = os.path.join(output_base, 'labels')
    output_video_dir = os.path.join(output_base, 'output')

    os.makedirs(image_out_dir, exist_ok=True)
    os.makedirs(label_out_dir, exist_ok=True)
    os.makedirs(output_video_dir, exist_ok=True)

    for video_file in tqdm(sorted(os.listdir(video_dir))):
        if not video_file.lower().endswith(".mp4"):
            continue

        video_id = os.path.splitext(video_file)[0]
        video_path = os.path.join(video_dir, video_file)
        output_video_path = os.path.join(output_video_dir, f"{video_id}.mp4")

        print(f"\n========== STARTED: {video_id} ==========")
        cap, total_frames, fps, w, h = setup_video_capture(video_path)
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        writer = cv2.VideoWriter(output_video_path, fourcc, fps, (w, h))

        frame_count = 0
        pbar = tqdm(total=max_frames if max_frames else total_frames, desc=video_id)

        while True:
            ret, frame = cap.read()
            if not ret or (max_frames and frame_count >= max_frames):
                break

            annotated_bgr, boxes, class_ids = process_frame(
                frame, yolo_model, w, h, class_id_to_name, underrepresented_class_ids)

            # ✅ Save original image
            img_filename = f'{video_id}_{frame_count:04d}.jpg'
            img_path = os.path.join(image_out_dir, img_filename)
            cv2.imwrite(img_path, frame)

            # ✅ Save YOLO-format label (only if boxes found)
            if boxes:
                label_filename = f'{video_id}_{frame_count:04d}.txt'
                label_path = os.path.join(label_out_dir, label_filename)
                with open(label_path, 'w') as f:
                    for box, cls_id in zip(boxes, class_ids):
                        x1, y1, x2, y2 = box
                        w_box = x2 - x1
                        h_box = y2 - y1
                        cx = x1 + w_box / 2
                        cy = y1 + h_box / 2
                        f.write(f"{int(cls_id)} {cx/w:.6f} {cy/h:.6f} {w_box/w:.6f} {h_box/h:.6f}\n")

            writer.write(annotated_bgr)
            frame_count += 1
            pbar.update(1)

        cap.release()
        writer.release()
        pbar.close()
        print(f"DONE: {video_id} — Processed {frame_count} frames")


### Get Class Instances

In [160]:
import os
from collections import defaultdict

def get_rare_class_ids(label_dir, class_id_to_name, rare_threshold=1000):
    class_counts = defaultdict(int)

    for label_file in os.listdir(label_dir):
        if not label_file.endswith('.txt'):
            continue
        with open(os.path.join(label_dir, label_file), 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 1:
                    cls_id = int(parts[0])
                    class_counts[cls_id] += 1

    print("🔍 Class-wise instance counts:")
    for cls_id in sorted(class_counts.keys()):
        name = class_id_to_name.get(cls_id, ("Unknown", []))[0]
        count = class_counts[cls_id]
        print(f"Class {cls_id:2d} ({name:10s}): {count} instances")

    rare_class_ids = {cls_id for cls_id, count in class_counts.items() if count < rare_threshold}
    print(f"\n✅ Rare class IDs (threshold < {rare_threshold}): {rare_class_ids}")

    return rare_class_ids

### Merge predciton and previous datatsets

In [161]:
import os
import shutil

def merge_yolo_datasets(source1, source2, destination):
    # Define subfolders
    img1_dir = os.path.join(source1, 'images')
    lbl1_dir = os.path.join(source1, 'labels')
    img2_dir = os.path.join(source2, 'images')
    lbl2_dir = os.path.join(source2, 'labels')
    dst_img_dir = os.path.join(destination, 'images')
    dst_lbl_dir = os.path.join(destination, 'labels')

    # Create destination folders
    os.makedirs(dst_img_dir, exist_ok=True)
    os.makedirs(dst_lbl_dir, exist_ok=True)

    def copy_files(src_img_dir, src_lbl_dir, prefix):
        for filename in sorted(os.listdir(src_img_dir)):
            if not filename.lower().endswith('.jpg'):
                continue
            base = os.path.splitext(filename)[0]

            # Image
            new_img_name = f"{prefix}_{base}.jpg"
            shutil.copy(os.path.join(src_img_dir, filename),
                        os.path.join(dst_img_dir, new_img_name))

            # Label
            label_file = base + ".txt"
            if os.path.exists(os.path.join(src_lbl_dir, label_file)):
                new_lbl_name = f"{prefix}_{base}.txt"
                shutil.copy(os.path.join(src_lbl_dir, label_file),
                            os.path.join(dst_lbl_dir, new_lbl_name))
            else:
                print(f"⚠️ No label for {filename}")

    print("🔁 Merging original dataset...")
    copy_files(img1_dir, lbl1_dir, prefix="orig")

    print("➕ Merging predicted video dataset...")
    copy_files(img2_dir, lbl2_dir, prefix="pred")

    print(f"\n✅ Merge complete! Merged dataset at: {destination}")

### Print Metrics 

In [162]:
def find_results_csv(directory):
    """Find the results.csv file in the specified directory."""
    for root, dirs, files in os.walk(directory):
        if 'results.csv' in files:
            return os.path.join(root, 'results.csv')
    return None

def load_results_csv(results_csv_path):
    """Load the results CSV into a pandas DataFrame."""
    return pd.read_csv(results_csv_path)

def calculate_total_epochs(df):
    """Calculate the total number of epochs from the DataFrame."""
    return df['epoch'].max()

def calculate_training_loss(epoch_data):
    """Calculate the total training loss from the given epoch data."""
    train_box_loss = epoch_data['train/box_loss']
    train_cls_loss = epoch_data['train/cls_loss']
    train_dfl_loss = epoch_data['train/dfl_loss']
    return train_box_loss + train_cls_loss + train_dfl_loss

def calculate_validation_loss(epoch_data):
    """Calculate the total validation loss from the given epoch data."""
    val_box_loss = epoch_data['val/box_loss']
    val_cls_loss = epoch_data['val/cls_loss']
    val_dfl_loss = epoch_data['val/dfl_loss']
    return val_box_loss + val_cls_loss + val_dfl_loss

def print_final_metrics(df):
    """Print the final metrics for the last epoch."""
    final_epoch_data = df.iloc[-1]

    # Calculate total training and validation loss
    train_loss = calculate_training_loss(final_epoch_data)
    val_loss = calculate_validation_loss(final_epoch_data)

    # Print overall metrics
    print("\n========== Final Training Metrics ==========")
    print(f"Training Loss: {train_loss:.6f}")
    print(f"Precision: {final_epoch_data['metrics/precision(B)']:.6f}")
    print(f"Recall: {final_epoch_data['metrics/recall(B)']:.6f}")
    print(f"mAP@0.5: {final_epoch_data['metrics/mAP50(B)']:.6f}")
    print(f"mAP@0.5:0.95: {final_epoch_data['metrics/mAP50-95(B)']:.6f}")

    print("\n========== Final Validation Metrics ==========")
    print(f"Validation Loss: {val_loss:.6f}")


def print_csv_metrics(directory):
    """Main function to process and print final metrics."""
    # Find the results.csv file
    results_csv_path = find_results_csv(directory)
    
    if not results_csv_path:
        print("Error: 'results.csv' file not found in the specified directory.")
        return

    print(f"Found results.csv at: {results_csv_path}")

    # Load results CSV
    df = load_results_csv(results_csv_path)

    # Get the total number of epochs
    total_epochs = calculate_total_epochs(df)
    print(f"Total number of epochs: {total_epochs}")

    # Print columns in the CSV
    # print("\n========== Columns in CSV ==========")
    # print(df.columns)

    # Print final metrics
    print_final_metrics(df)


## Calling Functions

### Download, convert, split and normalize datsets 

In [163]:
# gdrive_url = "https://drive.google.com/file/d/1UppumYqYOi-kto6BWPfFxwJK2Eph46oY/view?usp=sharing"
# semantic_drone_dataset_download(gdrive_url, extract_to="datasets")

# gdrive_url = "https://drive.google.com/file/d/12cbrTaBAMIsuU-mwAA7IgDk9wSLC9cC-/view?usp=sharing"
# uavdt_dataset_download(gdrive_url, extract_to="datasets")

In [164]:
# dataset_path = "./datasets/semantic_drone_dataset/training_set"
# output_dir = "./datasets/semantic_yolo"

# convert_fulldataset_yolo_only(dataset_path, output_dir, class_id_to_name)

In [165]:
# #UAVDT-2024

# source_root = "./datasets/UAVDT-2024"
# output_root = "./datasets/new_dataset_yolo_split"

# convert_dataset(source_root)
# copy_split_sequences(source_root, output_root, train_ratio=0.8)


# # Semantic dorne datasets
# split_and_move_dataset()


In [166]:
# # Set your paths
# dataset_path = "./datasets/new_dataset_yolo_split/train"
# image_dir = os.path.join(dataset_path, "images")
# annotations_dir = os.path.join(dataset_path, "labels")

# normalize_all_labels(annotations_dir, image_dir)

# dataset_path = "./datasets/new_dataset_yolo_split/val"
# image_dir = os.path.join(dataset_path, "images")
# annotations_dir = os.path.join(dataset_path, "labels")

# normalize_all_labels(annotations_dir, image_dir)

### Checking how many classes have how much instances

In [167]:
# labels_dir = './datasets/new_dataset_yolo_split/train/labels'

# rare_class_ids = get_rare_class_ids(label_dir=labels_dir, class_id_to_name=class_id_to_name ,rare_threshold=3000)

### Training v8 model

In [168]:
# import shutil
# import os

# # List of folders to delete
# folders_to_delete = ['./datasets/semantic_yolo', './datasets/new_dataset_yolo', './datasets/uavdt-processed', './runs', "./metrics"]

# for folder_path in folders_to_delete:
#     if os.path.exists(folder_path):
#         shutil.rmtree(folder_path)
#         print(f"✅ Deleted folder: {folder_path}")
#     else:
#         print(f"⚠️ Folder does not exist: {folder_path}")


In [169]:
# print("[+] Training Start")

# gc.collect()
# torch.cuda.empty_cache()

# train_yolo("yolov8.yaml", 80, 720, 8, "yolov8", False, "")

### Print metrics

In [170]:
yolov8 = './runs/train/yolov8'
best_pt_path = find_best_model(yolov8)
# evaluate_and_save_metrics(best_pt_path)

✅ Found best.pt at: runs\train\yolov8\weights\best.pt


In [171]:
print_per_class_metrics("per_class_metrics.json")

📊 Per-Class mAP@0.5:0.95 Metrics:

Class Name      Class ID   mAP@0.5:0.95   
----------------------------------------
unlabeled       0          0.3008         
pool            1          0.8458         
vegetation      2          0.1329         
roof            3          0.5681         
wall            4          0.1462         
window          5          0.2316         
person          6          0.2769         
dog             7          0.4108         
car             8          0.3159         
bicycle         9          0.3551         
tree            10         0.4516         
truck           11         0.0336         
bus             12         0.0            
vehicle         13         0.1412         


In [172]:
print_csv_metrics(yolov8)

Found results.csv at: ./runs/train/yolov8\results.csv
Total number of epochs: 80

========== Final Training Metrics ==========
Training Loss: 1.535840
Precision: 0.535960
Recall: 0.359830
mAP@0.5: 0.411080
mAP@0.5:0.95: 0.289710

========== Final Validation Metrics ==========
Validation Loss: 5.173490


### Prediciton videso

In [173]:
# import shutil
# import os

# # List of folders to delete
# folders_to_delete = ['./datasets/new-videos-predicted-yolo', "./datasets/merged_yolo_dataset"]

# for folder_path in folders_to_delete:
#     if os.path.exists(folder_path):
#         shutil.rmtree(folder_path)
#         print(f"✅ Deleted folder: {folder_path}")
#     else:
#         print(f"⚠️ Folder does not exist: {folder_path}")


In [174]:
# process_all_videos(best_pt_path, class_id_to_name, rare_class_ids, video_dir='videos', output_base='./datasets/new-videos-predicted-yolo', max_frames=None)

In [175]:
# # Folder with YOLO label files
# label_dir = './datasets/new-videos-predicted-yolo/labels'

# get_rare_class_ids(label_dir=label_dir, class_id_to_name=class_id_to_name ,rare_threshold=0)


### Merge previous and new prediction Ddatasets

In [176]:

# merge_yolo_datasets(
#     source1='./datasets/new_dataset_yolo_split/train',
#     source2='./datasets/new-videos-predicted-yolo',
#     destination='./datasets/merged_yolo_dataset'
# )


In [177]:
# # Folder with YOLO label files
# label_dir = './datasets/merged_yolo_dataset/labels'

# rare_class_ids = get_rare_class_ids(label_dir=label_dir, class_id_to_name=class_id_to_name ,rare_threshold=0)



### Retrain Model on predictions

In [178]:
# import shutil
# import os
# import glob

# # Match all folders starting with 'fine-tune-yolov8' inside './runs_yolo/train/'
# folders_to_delete = glob.glob('./runs/train/fine-tune-yolov8*')

# for folder_path in folders_to_delete:
#     if os.path.isdir(folder_path):
#         shutil.rmtree(folder_path)
#         print(f"✅ Deleted folder: {folder_path}")
#     else:
#         print(f"⚠️ Not a directory or doesn't exist: {folder_path}")


In [179]:
# train_yolo("yolo_retrain.yaml", 10, 720, 8, "fine-tune-yolov8", True, dir=best_pt_path)

### Compare Models

In [180]:
new_path = './runs/train/fine-tune-yolov8'
print_csv_metrics(new_path)

Found results.csv at: ./runs/train/fine-tune-yolov8\results.csv
Total number of epochs: 10

========== Final Training Metrics ==========
Training Loss: 2.290150
Precision: 0.632960
Recall: 0.548170
mAP@0.5: 0.621000
mAP@0.5:0.95: 0.526450

========== Final Validation Metrics ==========
Validation Loss: 2.578870


In [181]:
import pandas as pd

def compare_final_metrics(csv1_path, csv2_path):
    # Load both result CSVs
    df1 = pd.read_csv(csv1_path)
    df2 = pd.read_csv(csv2_path)
    # print(df1.head())
    # Use the final row (last epoch)
    last1 = df1.iloc[-1]
    last2 = df2.iloc[-1]

    metrics_to_compare = {
        "train/box_loss": "Box Loss (Train)",
        "train/cls_loss": "Cls Loss (Train)",
        "train/dfl_loss": "DFL Loss (Train)",
        "metrics/precision(B)": "Precision",
        "metrics/recall(B)": "Recall",
        "metrics/mAP50(B)": "mAP@0.5",
        "metrics/mAP50-95(B)": "mAP@0.5:0.95",
        "val/box_loss": "Box Loss (Val)",
        "val/cls_loss": "Cls Loss (Val)",
        "val/dfl_loss": "DFL Loss (Val)"
    }

    print("🔍 Comparison of Final Epoch Metrics:\n")
    for key, label in metrics_to_compare.items():
        val1 = last1[key]
        val2 = last2[key]
        trend = "✅ Good Increase" if val2 > val1 else "❌ No Increase"
        print(f"{label:20s}: {val1:.5f} → {val2:.5f} | {trend}")

# Example usage


In [182]:
new_path = './runs/train/fine-tune-yolov8'
old_path = './runs/train/yolov8'
results_csv_path = find_results_csv(new_path)
results_csv_path_1 = find_results_csv(old_path)



compare_final_metrics(results_csv_path_1, results_csv_path)


🔍 Comparison of Final Epoch Metrics:

Box Loss (Train)    : 0.46372 → 0.75401 | ✅ Good Increase
Cls Loss (Train)    : 0.25350 → 0.66127 | ✅ Good Increase
DFL Loss (Train)    : 0.81862 → 0.87487 | ✅ Good Increase
Precision           : 0.53596 → 0.63296 | ✅ Good Increase
Recall              : 0.35983 → 0.54817 | ✅ Good Increase
mAP@0.5             : 0.41108 → 0.62100 | ✅ Good Increase
mAP@0.5:0.95        : 0.28971 → 0.52645 | ✅ Good Increase
Box Loss (Val)      : 1.76688 → 0.76374 | ❌ No Increase
Cls Loss (Val)      : 2.36712 → 0.94291 | ❌ No Increase
DFL Loss (Val)      : 1.03949 → 0.87222 | ❌ No Increase


In [183]:
best_pt_path = find_best_model(new_path)
# evaluate_and_save_metrics(best_pt_path, output_json_path="per_class_metrics_retrain.json")

✅ Found best.pt at: runs\train\fine-tune-yolov8\weights\best.pt


In [184]:
print_per_class_metrics("per_class_metrics_retrain.json")

📊 Per-Class mAP@0.5:0.95 Metrics:

Class Name      Class ID   mAP@0.5:0.95   
----------------------------------------
road            0          0.5266         
pool            1          0.5266         
vegetation      2          0.5266         
roof            3          0.8812         
wall            4          0.6481         
window          5          0.7973         
person          6          0.5386         
dog             7          0.0983         
car             8          0.5266         
bicycle         9          0.3785         
tree            10         0.7665         
truck           11         0.0            
bus             12         0.517          
vehicle         13         0.6401         


In [185]:
import json

def compare_maps(json_path1, json_path2):
    with open(json_path1, 'r') as f1, open(json_path2, 'r') as f2:
        metrics1 = json.load(f1)
        metrics2 = json.load(f2)

    print("\n📊 Comparison of mAP@0.5:0.95 per class:\n")
    print(f"{'Class':<15} {'Before':<10} {'After':<10} {'Change'}")
    print("-" * 50)

    for class_name in metrics1:
        map1 = metrics1[class_name].get("mAP@0.5:0.95", 0)
        map2 = metrics2.get(class_name, {}).get("mAP@0.5:0.95", 0)

        if map2 > map1:
            status = "✅ Good increase"
        else:
            status = "❌ No increase"

        print(f"{class_name:<15} {map1:<10.4f} {map2:<10.4f} {status}")

# 🔧 Example usage:
compare_maps("per_class_metrics.json", "per_class_metrics_retrain.json")



📊 Comparison of mAP@0.5:0.95 per class:

Class           Before     After      Change
--------------------------------------------------
unlabeled       0.3008     0.0000     ❌ No increase
pool            0.8458     0.5266     ❌ No increase
vegetation      0.1329     0.5266     ✅ Good increase
roof            0.5681     0.8812     ✅ Good increase
wall            0.1462     0.6481     ✅ Good increase
window          0.2316     0.7973     ✅ Good increase
person          0.2769     0.5386     ✅ Good increase
dog             0.4108     0.0983     ❌ No increase
car             0.3159     0.5266     ✅ Good increase
bicycle         0.3551     0.3785     ✅ Good increase
tree            0.4516     0.7665     ✅ Good increase
truck           0.0336     0.0000     ❌ No increase
bus             0.0000     0.5170     ✅ Good increase
vehicle         0.1412     0.6401     ✅ Good increase


In [ ]:
new_path = './runs/train/fine-tune-yolov8'
best_pt_path = find_best_model(new_path)
videos_predictions(best_pt_path, class_id_to_name, video_dir='videos', output_base='./datasets/final_output', max_frames=None)

  0%|          | 0/9 [00:00<?, ?it/s]


========== STARTED: v1 ==========


v1:   0%|          | 0/642 [00:00<?, ?it/s]

DONE: v1 — Processed 642 frames

========== STARTED: v10 ==========


v10:   0%|          | 0/259 [00:00<?, ?it/s]

...

DONE: v6 — Processed 483 frames

========== STARTED: v8 ==========


v8:   0%|          | 0/949 [00:00<?, ?it/s]

DONE: v8 — Processed 949 frames

========== STARTED: v9 ==========


v9:   0%|          | 0/2752 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [187]:
best_pt_path = find_best_model(old_path)

videos_predictions(best_pt_path, class_id_to_name, video_dir='videos', output_base='./datasets/final_output_1', max_frames=None)


✅ Found best.pt at: runs\train\yolov8\weights\best.pt


  0%|          | 0/9 [00:00<?, ?it/s]


========== STARTED: v1 ==========


v1:   0%|          | 0/642 [00:00<?, ?it/s]

DONE: v1 — Processed 642 frames

========== STARTED: v10 ==========


...

DONE: v6 — Processed 483 frames

========== STARTED: v8 ==========


v8:   0%|          | 0/949 [00:00<?, ?it/s]

DONE: v8 — Processed 949 frames

========== STARTED: v9 ==========


v9:   0%|          | 0/2752 [00:00<?, ?it/s]

DONE: v9 — Processed 2752 frames
